## Yelp Dataset Recommender System - Data Wrangling and Modelling
by Saran Liukasemsarn, Elliot Smalling and Sachin Sridhar

### Part 1: Data Wrangling

In the EDA portion of the projects we decided to filter the dataset to contain only restaurants in Canada. This abbreviated dataset is located [here](https://drive.google.com/open?id=1r5K6-7MWacaLjp25nlYL8DtazJ8aDh7E). Based on the results of the EDA and the design decisions made for the analysis, we perform data wrangling, the steps of which have been outlined below -
1. Eliminate restaurants with a low number of reviews (for reliability of information)
2. Eliminate users with a low number of restaurant reviews
3. Perform one-hot encoding for users and restaurants
4. Split the reviews into three datasets randomly: training, test, and holdout, such that every restaurant is in the training set and that every user is in each dataset

In [1]:
# import relevant libraries

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import json
import copy
from collections import Counter
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
import pandas as pd
import datetime
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
%matplotlib inline

Data has already been filtered based on geographical location (only restaurants in two states of Canada - Ontario and Quebec - have been selected) and basic data cleaning operations have been carried out on the datasets shared by Yelp. The modified (cleaned) versions of the three datasets have been imported below.

In [2]:
# Importing the datasets
business = pd.read_csv('cleaned_data/business.csv')
review = pd.read_csv('cleaned_data/review.csv')
user = pd.read_csv('cleaned_data/user.csv')

In [3]:
# Excluding an unnecessary column from the three datasets
if 'Unnamed: 0' in business.columns:
    del business['Unnamed: 0']
if 'Unnamed: 0' in review.columns:
    del review['Unnamed: 0']
if 'Unnamed: 0' in user.columns:
    del user['Unnamed: 0']

Let us create a table showing the number of users with n reviews for all n in the review dataset (Table showing number of users that make a particular number of reviews).

In [4]:
# get number of reviews for each user and sort these sizes
review_user = review.groupby(['user_id']).size().sort_values()
# make a new dataframe
review_frame = pd.DataFrame(review_user.groupby(review_user).size())
review_frame.columns = ['number of users with n reviews']
review_frame['number of reviews (n)'] = review_frame.index
review_frame = review_frame[['number of reviews (n)','number of users with n reviews']]
review_frame = review_frame.reset_index(drop=True)
review_frame.head(10)

,number of reviews (n),number of users with n reviews
0,1,56926
1,2,19261
2,3,9854
3,4,6124
4,5,3969
5,6,2763
6,7,2091
7,8,1633
8,9,1234
9,10,1018


In [5]:
review_frame.tail(2)

,number of reviews (n),number of users with n reviews
249,545,1
250,2139,1


From the above table, it is evident that there are many users with very few reviews. These could lead to unreliable predictions due to lack of sufficient information, and hence, we will eliminate them. We will also remove restaurants with very few reviews. For example, if we have just two reviews for a restaurant, we cannot be sure that the sources of the two reviews are reliable.

**Restaurants with fewer than 15 reviews and users with fewer than 50 reviews have been excluded from the analysis, in order to enhance reliability and improve the accuracy of our results.**

In [6]:
review2 = (review.groupby('business_id').filter(lambda g: len(g) > 15))
review2 = (review2.groupby('user_id').filter(lambda g: len(g) > 50)).copy(deep=True).reset_index(drop=True)
review2_backup = review2.copy(deep=True)

Below is a summary of the dataframe *review2*:

In [7]:
print('# users with more than 50 reviews (review2): ', np.sum(review2.groupby(['user_id']).size() > 50))
print('total # users (review2):                     ', len(review2.groupby(['user_id'])))
print('total # restaurants (review2):               ', len(review2.groupby(['business_id'])))
print('total # reviews (review2):                   ', len(review2))

# users with more than 50 reviews (review2):  1073
total # users (review2):                      1073
total # restaurants (review2):                7345
total # reviews (review2):                    103112


The total number of users with more than 50 reviews equals the total number of users in review2. Now, let's generate user2 and business2.

In [8]:
# get all the users in review2
list_users = list(review2['user_id'].unique())
# get all the restaurants in review2
list_restaurants = list(review2['business_id'].unique())
user2 = user[(user['user_id']).isin(list_users)]
business2 = business[(business['business_id']).isin(list_restaurants)]
print('total # users (user2):                       ', len(user2))
print('total # restaurants (business2):             ', len(business2))

total # users (user2):                        1073
total # restaurants (business2):              7345


The total number of users in *user2* equals the total number of users in *review2*. The total number of restaurants in *business2* also equals to the total number of restaurants in *review2*. 

**The next task is to split the data, and we start by permuting the data. The objective is to make sure that each user is present in every split and that the training set contains every restaurant in review2.** This must be done to ensure that we have some information about each user and that the matrix in Part 3 covers all the possibilities. 

In [9]:
# set seed
np.random.seed(9001)
# permute the data
review2 = review2_backup.copy(deep=True)
permuted_index = np.random.permutation(review2.index)
review2 = review2.reindex(permuted_index)

In [10]:
# create a temporary column that will be used to split the data
review2['temp'] = range(len(review2))

# group by restaurant and rank them
ranks_rest = review2.groupby('business_id')['temp'].rank(method='first')

# delete temp variable which is no longer important
del review2['temp']

# split the data, part1 contains the first observation of each restaurant
part1 = (review2[ranks_rest == 1]).copy(deep=True)
# part 2 is the rest
part2 = (review2[ranks_rest != 1]).copy(deep=True)

In [11]:
part2['temp'] = range(len(part2))

# ranks assigned to observations according to order they appear in the dataframe (for each group)
ranks_user = part2.groupby('user_id')['temp'].rank(method='first')

# counts
counts_user = part2['user_id'].map(part2.groupby('user_id').size())

train_mask = (ranks_user/counts_user < 0.15)
test_mask = (0.15 <= ranks_user/counts_user) & (ranks_user/counts_user < 0.3)
holdout_mask = (0.3 <= ranks_user/counts_user)

# delete the 'temp' variable -> it is no longer important
del part2['temp']

#### Since the variables *user_id* and *restaurant_id* are categorical variables, we convert them into their biniarized forms (create indicator variables) for each value taken up by these two variables.

In [12]:
# split
train_set = pd.concat([part1, part2[train_mask]], axis=0)
train_set['type'] = 'train'
test_set = (part2[test_mask]).copy(deep=True)
test_set['type'] = 'test'
holdout_set = (part2[holdout_mask]).copy(deep=True)
holdout_set['type'] = 'holdout'

review2 = pd.concat([train_set, test_set, holdout_set], axis=0)

    
# make dummy variables but keep original 'user_id' and 'business_id' in there, drop one binary variable of each type
review2 = pd.concat([review2, pd.get_dummies(review2[['user_id','business_id']], drop_first=True)], axis=1)

In [13]:
train_set = review2[review2['type'] == 'train']
test_set = review2[review2['type'] == 'test']
holdout_set = review2[review2['type'] == 'holdout']

In [14]:
print('size of training set: ', len(train_set))
print('size of test set:     ', len(test_set))
print('size of holdout set:  ', len(holdout_set))

compare_data_sets = pd.DataFrame()
compare_data_sets['training'] = train_set.groupby('user_id').size()
compare_data_sets['test'] = test_set.groupby('user_id').size()
compare_data_sets['holdout'] = holdout_set.groupby('user_id').size()

compare_data_sets.head()

size of training set:  21154
size of test set:      14331
size of holdout set:   67627


,training,test,holdout
user_id,,,
--Qh8yKWAvIP4V4K8ZPfHA,38,27,124
-B4Cf2XLkPr9qMlLPHJAlw,12,9,45
-KVxkJDSTjtPGsamMDG92Q,17,11,52
-KpEgEen1tj-jdjIS7uVOw,23,8,37
-RCD8F7qbsLfzT3k1HtMxg,12,9,43


We make two observations from the outputs displayed above. First, we notice the split between the training, test and the hold-out datasets. Second, in the table displayed above, we find a description of how the reviews of each user have been split across the three datasets. It has been ensured that the reviews of a user are well represented in all the three datasets - the training, test and hold-out.

In [15]:
print('number of unique users:                           ', len(review2['user_id'].unique()))
print('number of unique users in the training set:       ', len(train_set['user_id'].unique()))
print('number of unique users in the test set:           ', len(test_set['user_id'].unique()))
print('number of unique users in the holdout set:        ', len(holdout_set['user_id'].unique()))
print('number of unique restaurants:                     ', len(review2['business_id'].unique()))
print('number of unique restaurants in the training set: ', len(train_set['business_id'].unique()))

number of unique users:                            1073
number of unique users in the training set:        1073
number of unique users in the test set:            1073
number of unique users in the holdout set:         1073
number of unique restaurants:                      7345
number of unique restaurants in the training set:  7345


In [16]:
train_set_x = train_set.copy(deep=True)
test_set_x = test_set.copy(deep=True)
holdout_set_x = holdout_set.copy(deep=True)

for var in ['type','business_id', 'date', 'stars', 'user_id']:
    if var in train_set_x.columns:
        del train_set_x[var]
    if var in test_set_x.columns:
        del test_set_x[var]
    if var in holdout_set_x.columns:
        del holdout_set_x[var]

train_set_y = train_set['stars']
test_set_y = test_set['stars']
holdout_set_y = holdout_set['stars']

In [17]:
# Getting a view of the predictors in the training dataset
# These are the binarized versions of each user_id and restaurant_id present in the dataset
train_set_x.head(2)

,user_id_-B4Cf2XLkPr9qMlLPHJAlw,user_id_-KVxkJDSTjtPGsamMDG92Q,user_id_-KpEgEen1tj-jdjIS7uVOw,user_id_-RCD8F7qbsLfzT3k1HtMxg,user_id_-_2h2cJlBOWAYrfplMU-Cg,user_id_-d2daWmftYumOaYpbD5D8Q,user_id_-dbWm5L_Ol2hZeLRoQOK7w,user_id_-fEe8XBeJ6pGLIeAyAWzfw,user_id_-hUgrj7Lzir3yLUYrMYQ4g,user_id_-m0KTRk0c901-4b-BN34Gg,...,business_id_zvtkeghW0Px5HY9QkJ4INw,business_id_zw4Legbcu018p5WcZ74iWA,business_id_zw74kL1IvT65yRvNLx5UxA,business_id_zwkif4XLEDqdEwEgTWLIVQ,business_id_zxJlg4XCHNoFy78WZPv89w,business_id_zy_NHTqtfSrfTGGPoqy4Mw,business_id_zyw5DjrRks7a8OhmBsgCQQ,business_id_zz3CqZhNx2rQ_Yp6zHze-A,business_id_zze6IysT7bJFS8gvi6fZ2A,business_id_zzlZJVkEhOzR2tJOLHcF2A
2721,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
39119,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Part 2(a): Creating a Basline Linear Regression Model
In the first part of our analysis, we create a baseline estimate of the ratings.
#### We fit the baseline linear regression model, represented mathematically as $\hat{Y}_{um} = \hat{\mu} + \hat{\theta}_u + \hat{\gamma}_m$

In [18]:
# Fitting the linear baseline regression model
regress1 = LinearRegression()
regress1.fit(train_set_x, train_set_y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [19]:
print('test score 1:    ', regress1.score(test_set_x, test_set_y))

test score 1:     -0.130286650544


We notice a poor score for the Test/Validation $R^2$, and hence performed Regulzarized Regression (with Lasso).

### Part 2(b): Creating a Regularized Regression

In [20]:
lambdas = [.00001, .0001, .001,.005,1,5,10,50,100,500,1000]
regress2 = LassoCV(cv=10, alphas=lambdas)
regress2.fit(train_set_x, train_set_y)

LassoCV(alphas=[1e-05, 0.0001, 0.001, 0.005, 1, 5, 10, 50, 100, 500, 1000],
    copy_X=True, cv=10, eps=0.001, fit_intercept=True, max_iter=1000,
    n_alphas=100, n_jobs=1, normalize=False, positive=False,
    precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
    verbose=False)

In [21]:
print('test score 2:    ', regress2.score(test_set_x, test_set_y))

test score 2:     0.0993635027725


In contrast to the Linear Regression Model without Regularization, we observe a significant improvement in the $R^2$ value on the test dataset. 

**The $R^2$ of the Lasso Regularized Model on the test dataset is 0.1**

**As a consequence of improved performance of the Lasso Regularized Regression Model, we choose this as our baseline, and perform all subsequent checks/comparisons with respect to this model.**

In [22]:
regress2.alpha_

0.0001

We use the residuals obtained from the regression model in the next stage of our analysis -

### Part 3: Matrix Factorization

The objective is to create a matrix, which contains residuals from the baseline model on the training dataset. The matrix will have the list of users along its rows and the list of restaurants along its columns.

This will be obtained by pivoting our existing dataframe appropriately. Also, the **matrix is sparse** i.e. most of the matrix values do not exist. We intially fill them in using zeros, and our goal is to find the *'best guess'* for these values through Matrix Factorization using Alternating Least Squares.

In order to perform computations in subsequent steps, we will need to replace the null values. Based on consideration of the Weight Matrix $W$ (explained below), we set these null values to 0. Setting the Null values to zero will not conflict with the calculated residuals, since none of the training residuals has been found to be 0.

In [23]:
# Find the predicted values for the training dataset from the lasso regualized model
y_hat = regress2.predict(train_set_x)
y_hat_test = regress2.predict(test_set_x)

# Residual = Observed Value - Predicted value
resid = train_set_y - y_hat
resid_test = test_set_y - y_hat_test

# Now, merge this array with X_train to get a dataset containing all the predictors and
# the corresponding residuals

In [24]:
train_set_resid = train_set.copy(deep = True) 
train_set_resid.shape

(21154, 8421)

In [25]:
# Including the additional column to the dataframe
train_set_resid['residuals'] = resid
train_set_resid.head()

,business_id,date,stars,user_id,type,user_id_-B4Cf2XLkPr9qMlLPHJAlw,user_id_-KVxkJDSTjtPGsamMDG92Q,user_id_-KpEgEen1tj-jdjIS7uVOw,user_id_-RCD8F7qbsLfzT3k1HtMxg,user_id_-_2h2cJlBOWAYrfplMU-Cg,...,business_id_zw4Legbcu018p5WcZ74iWA,business_id_zw74kL1IvT65yRvNLx5UxA,business_id_zwkif4XLEDqdEwEgTWLIVQ,business_id_zxJlg4XCHNoFy78WZPv89w,business_id_zy_NHTqtfSrfTGGPoqy4Mw,business_id_zyw5DjrRks7a8OhmBsgCQQ,business_id_zz3CqZhNx2rQ_Yp6zHze-A,business_id_zze6IysT7bJFS8gvi6fZ2A,business_id_zzlZJVkEhOzR2tJOLHcF2A,residuals
2721,bRmb81XDG3E2SOHARBLTog,2010-08-15,4,oBc0gQ4RpFrqzpNlH6_epA,train,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.455356
39119,X_Pg8SvGGYhCxwWRkrUv3Q,2016-01-24,4,dT1jqOZrFUmY4m4o37c8rw,train,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.279164
49343,8xI4hJ3nS4avEoo_l62dkw,2010-09-26,3,qOdmye8UQdqloVNE059PkQ,train,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-0.885521
83548,q9_gLvTNf11etVxbH7JY0Q,2017-01-26,4,Jm5h-bDATqRMWs3VahkFPg,train,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-0.403760
52893,J9BmILDpV1Pr3GKU9XhjTQ,2008-11-27,4,Yp7_GeD6KTRoo4Nteqv4SA,train,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.392325


In [26]:
train_set_resid.shape

(21154, 8422)

In [27]:
test_set_resid = test_set.copy(deep = True) 
test_set_resid['residuals'] = resid_test

** Before we pivot the data, we have ensured that the user_id and business_id combinedly describe the level of the table (implying that a person does not review the same restaurant multiple times).**

In [28]:
# Check if any user has reviewed a restaurant more than once
print('maximum number of reviews for a user-restaurant pair: ', max(train_set_resid.groupby(['user_id', 'business_id']).size().groupby(level=0).max()))

maximum number of reviews for a user-restaurant pair:  1


In [29]:
# Now we pivot this dataframe into the required format
# Users along rows, restaurants along columns, residuals as entries
df_review = train_set_resid.pivot(index = 'user_id', columns ='business_id', values = 'residuals').fillna(0)
df_review.head()

business_id,--6MefnULPED_I942VcFNA,--DaPTJW3-tB1vP-PfdTEg,--SrzpvFLwP_YFwB_Cetow,-0CTrPQNiSyClxhdO4HSDQ,-0DET7VdEQOJVJ_v6klEug,-0NhdsDJsdarxyDPR523ZQ,-0NrB58jqKqJfuUCDupcsw,-0mm8pqBSIOYZQHeo8XnkA,-1xuC540Nycht_iWFeJ-dw,-25X5v1q3WU6s-craJSvTw,...,zvtkeghW0Px5HY9QkJ4INw,zw4Legbcu018p5WcZ74iWA,zw74kL1IvT65yRvNLx5UxA,zwkif4XLEDqdEwEgTWLIVQ,zxJlg4XCHNoFy78WZPv89w,zy_NHTqtfSrfTGGPoqy4Mw,zyw5DjrRks7a8OhmBsgCQQ,zz3CqZhNx2rQ_Yp6zHze-A,zze6IysT7bJFS8gvi6fZ2A,zzlZJVkEhOzR2tJOLHcF2A
user_id,,,,,,,,,,,,,,,,,,,,,
--Qh8yKWAvIP4V4K8ZPfHA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-B4Cf2XLkPr9qMlLPHJAlw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-KVxkJDSTjtPGsamMDG92Q,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-KpEgEen1tj-jdjIS7uVOw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-RCD8F7qbsLfzT3k1HtMxg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
# Converting the pivoted dataframe into a numpy matrix:
df_review_matrix = df_review.as_matrix() 

In [31]:
df_review_matrix.shape

(1073, 7345)

We started out with 1073 users and 7345 restaurants in the training dataset, so the dimensions of the matrix seem to be correct.

We setup to apply the **Alternating Least Squares Regression** method to accomplish matrix factorization. Our objective is to minimize the following loss function:

$\sum_{u,m}(Y_{u,m} - \mu -\bar{\theta}.I_{u} - \bar{\gamma}I_m-\bar{q}_m^T\bar{p}_u)^2 + \alpha(\theta^2 + \gamma_m^2 + ||\bar{q}_m||^2 + ||\bar{p}_u||^2)$

A brief description of the methodology has been shown below -

We perform the process of validation in order to tune the parameters alpha (penalty) and the number of latent factors in the matrix. Due to extremely large run-times, only two combinations of the validations have been shown below, however, multiple other combinations of *alpha* and number of latent factors have been explored/checked. 

We arrive at the optimal values of *alpha* and number of latent factors to be considered in the process of factorization by computing the sum of squared errors on the test set.

In [32]:
# Performing Alternating Least Squares Regression

#Setting things up:
# We define Matrix Q as the matrix of residuals
Q = df_review_matrix.copy()

# Based on Q, we define W as follows -
# Where the initial entires were non-zero, fill-in 1, and where the initial entires were 0,
# fill in 0
W = Q!=0
W[W == True] = 1
W[W == False] = 0
W = W.astype(np.float64, copy=False)
m, n = Q.shape
n_iterations = 5

In [33]:
#def get_error(Q, X, Y, W):
#    return np.sum((W * (Q - np.dot(X, Y)))**2)

#list_lambda_ = [0.05, 0.5, 5, 50]
#list_n_factors = [4,10]
#train_error = []
#test_sse = []

#for lambda_ in list_lambda_:
#    for n_factors in list_n_factors:
#        X = np.random.randn(m, n_factors) 
#        Y = np.random.randn(n_factors, n)
#        weighted_errors = []
#        print('********************')
#        print('lambda:    ', lambda_)
#        print('n_factors: ', n_factors)
#        for ii in range(n_iterations):
#            for u, Wu in enumerate(W):
#                X[u] = np.linalg.solve(np.dot(Y, np.dot(np.diag(Wu), Y.T)) + lambda_ * np.eye(n_factors),
#                               np.dot(Y, np.dot(np.diag(Wu), Q[u].T))).T
#            for i, Wi in enumerate(W.T):
#                Y[:,i] = np.linalg.solve(np.dot(X.T, np.dot(np.diag(Wi), X)) + lambda_ * np.eye(n_factors),
#                                 np.dot(X.T, np.dot(np.diag(Wi), Q[:, i])))
#            weighted_errors.append(get_error(Q, X, Y, W))
#            print('iteration '+ str(ii+1) + ' is completed')
#        weighted_Q_hat = np.dot(X,Y)
#        # training error
#        train_error.append(((lambda_, n_factors),weighted_errors))
#        preds_df = pd.DataFrame(weighted_Q_hat, columns = df_review.columns)
#        user_ids = np.array(df_review.index)
#        preds_df['user_id'] = user_ids
#        user_business_residuals = pd.melt(preds_df,id_vars = ['user_id'],value_vars = df_review.columns)
#        test_data = test_set_resid.copy(deep=True)
#        test_subset = test_data[['business_id','user_id','residuals']]
#        result_test = pd.merge(test_subset,user_business_residuals, on = ['user_id','business_id'], how = 'inner')
#        result_test['resid_of_resid'] = result_test['residuals'] - result_test['value']
#        test_sse.append(((lambda_, n_factors),np.sum(result_test['resid_of_resid']**2)))
        
#validation_results = pd.DataFrame(test_sse)
#validation_results.columns = ['lambda, n_factors', 'sum of squared errors']
#validation_results.to_csv('Val_Scores.csv', index=False)

In [34]:
val_scores = pd.read_csv('Val_Scores.csv')
val_scores

,lambda,n_factors,sum of squared errors
0,0.05,4,24192.03429
1,0.05,10,21338.07873
2,0.50,4,19508.64591
3,0.50,10,16860.38679
4,5.00,4,13330.57669
5,5.00,10,13275.59727
6,50.00,4,13215.75947
7,50.00,10,13215.75947


In [35]:
best_lambda_ = 50
best_n_factors = 4
X = np.random.randn(m, best_n_factors) 
Y = np.random.randn(best_n_factors, n)
print('********************')
print('lambda:    ', best_lambda_)
print('n_factors: ', best_n_factors)
for ii in range(n_iterations):
    for u, Wu in enumerate(W):
        X[u] = np.linalg.solve(np.dot(Y, np.dot(np.diag(Wu), Y.T)) + best_lambda_ * np.eye(best_n_factors),
                        np.dot(Y, np.dot(np.diag(Wu), Q[u].T))).T
    for i, Wi in enumerate(W.T):
        Y[:,i] = np.linalg.solve(np.dot(X.T, np.dot(np.diag(Wi), X)) + best_lambda_ * np.eye(best_n_factors),
                         np.dot(X.T, np.dot(np.diag(Wi), Q[:, i])))
    print('iteration '+ str(ii+1) + ' is completed')
weighted_Q_hat = np.dot(X,Y)
preds_df = pd.DataFrame(weighted_Q_hat, columns = df_review.columns)
user_ids = np.array(df_review.index)
preds_df['user_id'] = user_ids
user_business_residuals = pd.melt(preds_df,id_vars = ['user_id'],value_vars = df_review.columns)

********************
lambda:     50
n_factors:  4
iteration 1 is completed
iteration 2 is completed
iteration 3 is completed
iteration 4 is completed
iteration 5 is completed


In [36]:
train_data = train_set_resid.copy(deep=True)
train_subset = train_data[['business_id','user_id','residuals']]
result_train = pd.merge(train_subset,user_business_residuals, on = ['user_id','business_id'], how = 'inner')
result_train['resid_of_resid'] = result_train['residuals'] - result_train['value']
result_train = result_train.rename(columns={'value': 'residuals_from_mat_factrz', 
                        'residuals': 'residuals_from_linear_regr'})

result_train.head()

,business_id,user_id,residuals_from_linear_regr,residuals_from_mat_factrz,resid_of_resid
0,bRmb81XDG3E2SOHARBLTog,oBc0gQ4RpFrqzpNlH6_epA,0.455356,9.933523e-17,0.455356
1,X_Pg8SvGGYhCxwWRkrUv3Q,dT1jqOZrFUmY4m4o37c8rw,0.279164,-2.293740e-19,0.279164
2,8xI4hJ3nS4avEoo_l62dkw,qOdmye8UQdqloVNE059PkQ,-0.885521,-1.049530e-17,-0.885521
3,q9_gLvTNf11etVxbH7JY0Q,Jm5h-bDATqRMWs3VahkFPg,-0.403760,-1.918724e-17,-0.403760
4,J9BmILDpV1Pr3GKU9XhjTQ,Yp7_GeD6KTRoo4Nteqv4SA,0.392325,1.714463e-18,0.392325


In [37]:
train_subset.head()

,business_id,user_id,residuals
2721,bRmb81XDG3E2SOHARBLTog,oBc0gQ4RpFrqzpNlH6_epA,0.455356
39119,X_Pg8SvGGYhCxwWRkrUv3Q,dT1jqOZrFUmY4m4o37c8rw,0.279164
49343,8xI4hJ3nS4avEoo_l62dkw,qOdmye8UQdqloVNE059PkQ,-0.885521
83548,q9_gLvTNf11etVxbH7JY0Q,Jm5h-bDATqRMWs3VahkFPg,-0.403760
52893,J9BmILDpV1Pr3GKU9XhjTQ,Yp7_GeD6KTRoo4Nteqv4SA,0.392325


In [38]:
result_train.to_csv('training_residuals_of_residuals.csv', index=False)

### Part 4: Attempting Other Modelling Techniques

Below, we combine all the datasets, along with the residuals from the matrix factorization stage of the model. We also create several variables: day of the week the review was written, month the review was written, length of time the user was active on Yelp when review was written.

In [39]:
#user.yelping_since = pd.to_datetime(user.yelping_since)
#review.date = pd.to_datetime(review.date)
#review['day'] = review.date.dt.weekday_name
#review['month'] = review.date.dt.strftime('%B')
#review = review.merge(user[['user_id','yelping_since']], how='left')
#review['time_yelping'] = (review.date-review.yelping_since).dt.days
#review.time_yelping[review.time_yelping < 1] = 1
#review['log_time_yelping'] = np.log(review.time_yelping)
#review['log_time_yelping2'] = review.log_time_yelping**2
resids = result_train
#resids = resids.merge(review, how='left')
resids = resids.merge(business[['business_id','city','state','latitude','longitude']], how='left')
resids.columns

Index(['business_id', 'user_id', 'residuals_from_linear_regr',
       'residuals_from_mat_factrz', 'resid_of_resid', 'city', 'state',
       'latitude', 'longitude'],
      dtype='object')

For each of the combinations of variables below, we fit a regular regression, a ridge regression, and a random forest regression as some models may be better suited to certain types of problems. We then perform 10-fold cross validation to assess how well the model woud perform out of sample

#### Model 1: day and month that the review was written, and time user was active on Yelp

In [40]:
# format data
#x_train = pd.get_dummies(resids[['day','month']]).iloc[:,1:18]
#x_train['log_time_yelping'] = resids.log_time_yelping
#x_train['log_time_yelping2'] = resids.log_time_yelping2
#y_train = resids.resid_of_resid

In [41]:
# simple linear regression
#lmdl = LinearRegression()
#lmdl.fit(x_train, y_train)
#print("10-fold CV R2:", cross_val_score(lmdl, x_train, y_train, cv=10).mean())

In [42]:
# ridge regression
#rmdl = RidgeCV()
#rmdl.fit(x_train, y_train)
#print("10-fold CV R2:", cross_val_score(rmdl, x_train, y_train, cv=10).mean())

In [43]:
# random forest regression
#depths = [2,3,5,7,10]
#for i in depths:
#    fmdl = RandomForestRegressor(n_estimators=100, max_features='sqrt', max_depth=i)
#    fmdl.fit(x_train, y_train)
#    print("10-fold CV R2, depth", i, ":", cross_val_score(fmdl, x_train, y_train, cv=10).mean())

#### Model 2: restaurant cities as factor variables

In [44]:
# format data
#x_train = pd.get_dummies(resids['city']).iloc[:,1:]
#y_train = resids.resid_of_resid

In [45]:
# simple linear regression
#lmdl = LinearRegression()
#lmdl.fit(x_train, y_train)
#print("10-fold CV R2:", cross_val_score(lmdl, x_train, y_train, cv=10).mean())

In [46]:
# ridge regression
#rmdl = RidgeCV()
#rmdl.fit(x_train, y_train)
#print("10-fold CV R2:", cross_val_score(rmdl, x_train, y_train, cv=10).mean())

In [47]:
# random forest regression
#depths = [2,3,5,7,10]
#for i in depths:
#    fmdl = RandomForestRegressor(n_estimators=100, max_features='sqrt', max_depth=i)
#    fmdl.fit(x_train, y_train)
#    print("10-fold CV R2, depth", i, ":", cross_val_score(fmdl, x_train, y_train, cv=10).mean())

#### Model 3: latitude and longitude interaction with state
Since restaurants in the dataset in Ontario and Quebec are really just located around Toronto and Montreal, respectively, we can view the coordinates' interaction with a state as an analysis of whether restauraunts in each city get higher ratings the farther west/east/north/south they are located in the city. For randomforest, the decision boundary becomes a literal geographic boundary, dividing the city up into sections and assigning score biases to each region.

In [48]:
# format data
x_train = (resids[['state','latitude','longitude']]).copy(deep=True)
x_train['QC'] = (x_train.state=='QC')*1
x_train['QC_latitude'] = x_train.QC*x_train.latitude
x_train['QC_longitude'] = x_train.QC*x_train.longitude
x_train = x_train.iloc[:,1:]
y_train = resids.resid_of_resid

In [49]:
# simple linear regression
lmdl = LinearRegression()
lmdl.fit(x_train, y_train)
print("10-fold CV R2:", cross_val_score(lmdl, x_train, y_train, cv=10).mean())

10-fold CV R2: -7.91065147508e-05


In [50]:
# ridge regression
rmdl = RidgeCV()
rmdl.fit(x_train, y_train)
print("10-fold CV R2:", cross_val_score(rmdl, x_train, y_train, cv=10).mean())

10-fold CV R2: -7.47857359459e-05


In [51]:
# random forest regression
depths = [2,3,5,7,10]
for i in depths:
    fmdl = RandomForestRegressor(n_estimators=100, max_features='sqrt', max_depth=i)
    fmdl.fit(x_train, y_train)
    print("10-fold CV R2, depth", i, ":", cross_val_score(fmdl, x_train, y_train, cv=10).mean())

10-fold CV R2, depth 2 : 0.00102056654732
10-fold CV R2, depth 3 : 0.00122405998344
10-fold CV R2, depth 5 : 0.000980687489795
10-fold CV R2, depth 7 : -0.00131674140843
10-fold CV R2, depth 10 : -0.0082639210523


#### Conclusion
We see above that the random forest model (with maximum tree depth of 5) fit on the latitude/longitude data performs the best out of the model tested. We now fit that model as our final model.

In [52]:
model3 = RandomForestRegressor(n_estimators=100, max_features='sqrt', max_depth=5)
model3.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [53]:
# predict using baseline regression
p1 = pd.Series(regress2.predict(holdout_set_x))

In [54]:
holdout_data = holdout_set.copy(deep = True) 
holdout_subset = holdout_data[['business_id','user_id']]
result_holdout = pd.merge(holdout_subset, user_business_residuals, on = ['user_id','business_id'], how = 'inner')
result_holdout = result_holdout.merge(business[['business_id','city','state','latitude','longitude']], how='left')
holdout_set_x3 = (result_holdout[['state','latitude','longitude']]).copy(deep=True)
holdout_set_x3['QC'] = (holdout_set_x3.state=='QC')*1
holdout_set_x3['QC_latitude'] = holdout_set_x3.QC*holdout_set_x3.latitude
holdout_set_x3['QC_longitude'] = holdout_set_x3.QC*holdout_set_x3.longitude
holdout_set_x3 = holdout_set_x3.iloc[:,1:]

In [55]:
p2 = result_holdout['value']

In [56]:
p3 = model3.predict(holdout_set_x3)

In [57]:
print('holdout r^2 for regression model:                                      ', r2_score(holdout_set_y, p1))
print('holdout r^2 for regression + matrix model:                             ', r2_score(holdout_set_y, p1+p2))
print('holdout r^2 for regression + matrix + Random Forest regression model:  ', r2_score(holdout_set_y, p1+p2+p3))

holdout r^2 for regression model:                                       0.0928745193673
holdout r^2 for regression + matrix model:                              0.0928745193673
holdout r^2 for regression + matrix + Random Forest regression model:   0.0972615487885


In [58]:
q1 = pd.Series(regress2.predict(train_set_x))
q2 = result_train['residuals_from_mat_factrz']
q3 = model3.predict(x_train)

In [59]:
print('training r^2 for regression model:                                      ', r2_score(train_set_y, q1))
print('training r^2 for regression + matrix model:                             ', r2_score(train_set_y, q1+q2))
print('training r^2 for regression + matrix + Random Forest regression model:  ', r2_score(train_set_y, q1+q2+q3))

training r^2 for regression model:                                       0.269712582918
training r^2 for regression + matrix model:                              0.269712582918
training r^2 for regression + matrix + Random Forest regression model:   0.27920343767


As these results show, the matrix factorization step does not improve the accuracy. On the other hand, the second regression does improve the accuracy. 